In [20]:
# Inspired by https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
from __future__ import print_function
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [8]:
raw_text = open('legfrance.txt').read()
text = raw_text[:1_000_000]
print('corpus length:', len(text))

corpus length: 1000000


In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 104


In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 333320


In [12]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [13]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [14]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
    print()

In [21]:
filepath="wi2-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [22]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
callbacks=[print_callback, checkpoint])

Epoch 1/60
333320/333320 [==============================] - 126s 377us/step - loss: 1.1323

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ode.
La Commission nationale d'aménageme"
ode.
La Commission nationale d'aménagement prévue à l'article L. 212-1 pour ou de l'aide et autogants de l'agrément de l'article L. 212-7.

Article 212-50

Les aides à l'article 212-20

Les aides financières et autogants de l'article L. 212-7.

Article 212-50

Les aides à l'article L. 212-7.

Article 211-50

Les aides à l'article L. 212-7.

Article 212-50

Les aides à l'article L. 212-7.

Article 232-22

Les aides à l'article L. 212-7 d----- diversity: 0.5
----- Generating with seed: "ode.
La Commission nationale d'aménageme"
ode.
La Commission nationale d'aménagement et autorisation et les dépenses présents à l'aide en astifiée de l'article 321-11-50 et autorisation conditions ou autorations sont comptes par l'allogration de l'autorisation d'assortissants d'appliquell

 droit d'entrée à chaque spectateur ou et le production audiovisuelle de la décision d'attribution d'œuvres cinématographiques ou une autre convention ou une procdment de ces délais de la décision d'attribution de l'œuvre au contrat de production et au moins au secteur du cinéma et de l'image animée artistique de l'aide est la production du pour la prestation du contrôle d'établissement de spectacles cinématographiques comportées pour l----- diversity: 1.0
----- Generating with seed: " droit d'entrée à chaque spectateur ou e"
 droit d'entrée à chaque spectateur ou en agestion ayant traisant pour la durée de premières tienser animée par 0.
Lorsquelle l'nusge ;

Alduration : jours pour la marit culaire de l'œuvre concirve le, conférent àvité par une letion des leurs averties sur lughier df production sur les résugés en conforde pour le renvoint versement lecturiciale du fait le qui 7 moins et du présent cas suivante :
1° Le montant prendre ou pour lorsque l'ar----- diversity: 1.2
----- G

/home/erwan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ématographique ou audiovisuelle de la commission de spectacle vivant des droits de projets d'œuvres de l'autorisation d'un document des dépenses de l'article 211-22 du code du cinéma et de l'image animée pour la projet d'attribution des aides financières autorisatiques de l'article L. 212-2.
Les aides financières autorisation des dépenses de projets d'œuvres de la commi----- diversity: 0.5
----- Generating with seed: "ors de l'autorisation préalable ou si de"
ors de l'autorisation préalable ou si des demandes d'art et de projets d'œuvres cinématographiques d'art et d'essai d'entreprise de production et autorisation des droits de contrôlés ;
15° La condition de projets d'œuvres immentues et le nombre de spectacles cinématographiques de longue durée et des médiations ;
14° Le dossier de conditiquteur de projets d'œuvres du ministre chargé de la culture de l'aide est attribuée par les projets d----- diversity: 1.0
----- Generating with seed: "ors de l'autorisation préalable ou si de"
ors 

icativement le récit par l'intégration de l'écriture des aprègection ou l'assurance de cafrchaccus de titre peur dans ce du réalisateur des investissemens, pour le producteur est effectué sur la compte d'autorisation de la responation de la commission des duréges contribuer notamment en version donnation et audiovisuel privé ;
8° monu d'un intéressefus d'unreche, ille le reconn énus avant liée et des dépenses de mboureres aux services f----- diversity: 1.2
----- Generating with seed: "icativement le récit par l'intégration d"
icativement le récit par l'intégration des aides à l'aide qui précisant les sociétés par le secteur audiovisuel de taatorme, le moyée pujsit à tent comprencer les conditions de dépôléaites ;
7° Cinsurel, catégorie plus totaliss d attribuée daux avances élenditaires apparaités sur le compte àentents considérées, le montant de l'article 6 200 du p)Ccagra littiré mentionné au Centre national du cinéma et de l'image animée ;
6° Les œuvresta

Epoch 00009: loss improved

...

 d'influence cinématographique, les diffuseurs de distribution d'une œuvre cinématographique de l'article L. 212-2 du code de commerce, à l'expert de décision de distribution et des dépenses de distribution d'une œuvre cinématographique de l'article L. 212-23 du code de commerce, à l'application des conditions de deux modifications des projets d'œuvres cinématographiques de longue durée dont en compte au montant de l'article L. 212-2 du----- diversity: 0.5
----- Generating with seed: " d'influence cinématographique, les diff"
 d'influence cinématographique, les diffuseurs et les personnalités d'attrateur présente les conditions précités et des distributeurs de constatation d'une autre entreprise de production de ce salles de spectation à la production de l'œuvre provisonné un ou les contrôles supérieux et le président du Centre national du cinéma et de l'image animée peuvent être informe de documentation des œuvres destinées à la diffusion d'une durée des dé----- diversity: 1.0
----- G

2° Les ont conclus avec la programmation du réception du réalisate de droit, barèpà nouveau groupement et im offriin du présent livre.

Un réalisateur ou habilité de ces affaires dans lesquelles il ;
6° Une production transfrontières garant se sélection valabliels et intervenils----- diversity: 1.2
----- Generating with seed: "tementale d'aménagement cinématographiqu"
tementale d'aménagement cinématographique et indurateur ou fai exajoutreques un établissement, les projets et choisis par la dortie lui pour demande sur chacunis à compter du leur respecter prévu ou des imeux, deuximdent paes précisant les conditions sui l xcraitée au moment du titre fixée des techniciens, présentembre secet à describsements peuvent résamacdé, nommélifimer la durée et de cours du cinéma et au diversige des imsgation
Art

Epoch 00030: loss did not improve from 0.75199
Epoch 31/60
333320/333320 [==============================] - 126s 379us/step - loss: 0.7515

----- Generating text after Epoch: 30
----- div

11° Le contrat de coproduction de la commission départementale d'aménagement cinématographique de l'aide total de la création de la commission des aides financières
cllocible et de la propriété intellectuelle de la déclaration de l'article L. 212-23 du code de cinéma et de l'image animée et des audiovisuels ;
------ diversity: 0.5
----- Generating with seed: "s sur ce visa.

Article L432-2

Dès la c"
s sur ce visa.

Article L432-2

Dès la coproduction cinématographique de l'aide de résident français ou d'un document équivalent d'un établissement de spectacles cinématographiques de la création cinématographique de l'aidec de la commission des aides financières
Et la propriété intellectuelle des barèmesions de la propriété internationale sont assimilées en vuent de l'application de l'article L. 212-23 et du présent livre.

Article 211----- diversity: 1.0
----- Generating with seed: "s sur ce visa.

Article L432-2

Dès la c"
s sur ce visa.

Article L432-2

Dès la coproduction de la droit 

Pour être consérêtrés après la propriété au cinéma estigent à ce dalibée du Ins au financement au moin à cette particulier en numenté des sraximements de l'autorisation.

Article Annexe 2-9
Les œuvres constituent sur son à récaccer inbsé----- diversity: 1.2
----- Generating with seed: "s le cadre du sous-programme MEDIA du pr"
s le cadre du sous-programme MEDIA du prodovisul ;
10° Annuelles
Article 211-30

La clause, dont le neuvindement ;
4° Tvcihe persevache :
1° Pour les sidents.
Sous-section 1 : Un nombre des prix de préparants, agent prarité lorsque l'œuvre cinématographique en tant 'im.
Evis pour la réalisation d'œuvres, titulaires de critires
par l'assurant ;
5° R. 331-32

309 400 000 € de 27 etisement e listior, fiscerment :
1° Sous-groupe “Paracoêter

Epoch 00037: loss did not improve from 0.75148
Epoch 38/60
333320/333320 [==============================] - 127s 380us/step - loss: 1.1435

----- Generating text after Epoch: 37
----- diversity: 0.2
----- Generating with seed: "r

KeyboardInterrupt: 

In [28]:
# load the network weights
filename = "wi2-31-0.7515.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               119296    
_________________________________________________________________
dense_1 (Dense)              (None, 104)               13416     
Total params: 132,712
Trainable params: 132,712
Non-trainable params: 0
_________________________________________________________________


In [24]:
13416/104

129.0